In [1]:
%%capture
import json
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
import numpy as np
import networkx as nx
import random
from gensim.models import Word2Vec

In [ ]:
FULL_SET_PATH = './data/aan_full.json'
TEST_SET_PATH = './data/aan_test.json'
DIRECTED_OUTPUT_PATH = './results/base_deepwalkcosine_directed_aan.json'
UNDIRECTED_OUTPUT_PATH = './results/base_deepwalkcosine_undirected_aan.json'

In [2]:
FULL_SET_PATH = './data/dblp_full.json'
TEST_SET_PATH = './data/dblp_test.json'
DIRECTED_OUTPUT_PATH = './results/base_deepwalkcosine_directed_dblp.json'
UNDIRECTED_OUTPUT_PATH = './results/base_deepwalkcosine_undirected_dblp.json'

In [3]:
with open(FULL_SET_PATH) as f:
    full_set = json.load(f)

directed_citation_graph = nx.DiGraph()
for paper in full_set:
    for ref_id in paper['references']:
        directed_citation_graph.add_edge(paper['id'], ref_id)
        
undirected_citation_graph = directed_citation_graph.to_undirected()

In [4]:
paper_ids = [paper['id'] for paper in full_set]

In [5]:
# inspire by https://github.com/phanein/deepwalk/

def deepwalk(G, number_walks=10, representation_size=64, seed=0, walk_length=40, window_size=5, workers=1):
    
    # build deepwalk corpus
    walks = []
    nodes = list(G.nodes())
    random.seed(seed)
  
    for _ in range(number_walks):
        random.shuffle(nodes)
        for node in nodes:
            # random walk
            path = [node]

            while len(path) < walk_length:
                current = path[-1]
                current_neighbors = list(G[current])
                if current_neighbors:
                    path.append(random.choice(current_neighbors))
                else:
                    break
                    
            #return [str(node) for node in path] 
            walks.append(path)

    # build model
    model = Word2Vec(walks, size=representation_size, window=window_size, min_count=0, sg=1, hs=1, workers=workers)    
    
    return model.wv

In [6]:
model_undirected = deepwalk(undirected_citation_graph)
model_directed = deepwalk(directed_citation_graph)

In [7]:
directed_paper_deepwalk_vectors = [model_directed.get_vector(paper['id']) for paper in full_set]
directed_paper_deepwalk_array = np.vstack(directed_paper_deepwalk_vectors)

In [8]:
undirected_paper_deepwalk_vectors = [model_undirected.get_vector(paper['id']) for paper in full_set]
undirected_paper_deepwalk_array = np.vstack(undirected_paper_deepwalk_vectors)

In [9]:
with open(TEST_SET_PATH) as f:
    test_set = json.load(f)

# directed variant

In [10]:
paper_deepwalk_array = directed_paper_deepwalk_array

partial_score_arrays = []
for i in tqdm(range(3)):
    partial_input_ids = [input_papers[i]['id'] for input_papers in test_set]
    input_deepwalk_vectors = [model_directed.get_vector(paper_id) for paper_id in partial_input_ids]
    
    input_deepwalk_array = np.vstack(input_deepwalk_vectors)

    partial_score_arrays.append(cosine_similarity(input_deepwalk_array, paper_deepwalk_array))

score_array = sum(partial_score_arrays)

In [11]:
results = []

for input_papers, scores  in tqdm(zip(test_set, score_array.tolist())):
    result = {}
    result['input'] = [input_paper['id'] for input_paper in input_papers]
    input_paper_ids_set = set(result['input'])
    
    candidate_scores = list(zip(paper_ids, scores))     
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    filtered_candidate_scores = [cs for cs in candidate_scores if cs[0] not in input_paper_ids_set]
    
    result['output'] = [cs[0] for cs in filtered_candidate_scores[:100]]
    results.append(result)

In [12]:
with open(DIRECTED_OUTPUT_PATH, 'w') as f:
    json.dump(results, f)

# undirected variant

In [13]:
paper_deepwalk_array = undirected_paper_deepwalk_array

partial_score_arrays = []
for i in tqdm(range(3)):
    partial_input_ids = [input_papers[i]['id'] for input_papers in test_set]
    input_deepwalk_vectors = [model_undirected.get_vector(paper_id) for paper_id in partial_input_ids]
    
    input_deepwalk_array = np.vstack(input_deepwalk_vectors)

    partial_score_arrays.append(cosine_similarity(input_deepwalk_array, paper_deepwalk_array))

score_array = sum(partial_score_arrays)

In [14]:
results = []

for input_papers, scores  in tqdm(zip(test_set, score_array.tolist())):
    result = {}
    result['input'] = [input_paper['id'] for input_paper in input_papers]
    input_paper_ids_set = set(result['input'])
    
    candidate_scores = list(zip(paper_ids, scores))     
    candidate_scores.sort(key=lambda x: x[1], reverse=True)
    filtered_candidate_scores = [cs for cs in candidate_scores if cs[0] not in input_paper_ids_set]
    
    result['output'] = [cs[0] for cs in filtered_candidate_scores[:100]]
    results.append(result)

In [15]:
with open(UNDIRECTED_OUTPUT_PATH, 'w') as f:
    json.dump(results, f)